In [1]:
#! pip install --upgrade https://github.com/niketanpansare/future_of_data/raw/master/systemml-1.3.0-SNAPSHOT-python.tar.gz

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190601181525-0000
KERNEL_ID = b330ce80-0be9-41f2-aac2-c44cc4d0c4e2
    100% |################################| 9.9MB 1.3MB/s eta 0:00:01
    100% |################################| 17.3MB 1.2MB/s eta 0:00:01
    100% |################################| 25.2MB 1.0MB/s eta 0:00:01
    100% |################################| 10.1MB 2.0MB/s eta 0:00:01
    100% |################################| 6.7MB 2.7MB/s eta 0:00:01
    100% |################################| 2.0MB 3.6MB/s eta 0:00:01
    100% |################################| 235kB 5.6MB/s eta 0:00:01
    100% |################################| 512kB 5.3MB/s eta 0:00:01
    100% |################################| 286kB 5.4MB/s eta 0:00:01
  Stored in directory: /home/spark/shared/.cache/pip/wheels/bf/b5/b7/b24122e11428907d1c632fc2e6fb0c7b85bba047f8dd263fde
Successfully built systemml
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which i

In [4]:
#!ln -s -f /home/spark/shared/user-libs/python3/systemml/systemml-java/systemml-1.3.0-SNAPSHOT-extra.jar ~/user-libs/spark2/systemml-1.3.0-SNAPSHOT-extra.jar
#!ln -s -f /home/spark/shared/user-libs/python3/systemml/systemml-java/systemml-1.3.0-SNAPSHOT.jar ~/user-libs/spark2/systemml-1.3.0-SNAPSHOT.jar
#!pip install mlxtend
!pip install python-imap

  Stored in directory: /home/spark/shared/.cache/pip/wheels/f2/61/7b/d853b983244afa4eb6ea30581ee0e7c6f858a0065c2640362e
Successfully built python-imap


In [6]:
# Disable Tensorflow from using GPU to avoid unnecessary evictions by SystemML runtime
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

# Import dependencies
from mlxtend.data import mnist_data
import numpy as np
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout,Flatten
from keras import backend as K
from keras.models import Model
from keras.optimizers import SGD
from systemml.mllearn import Keras2DML
# Set channel first layer
K.set_image_data_format('channels_first')

# Download the MNIST dataset
X, y = mnist_data()
X, y = shuffle(X, y)

# Split the data into training and test
n_samples = len(X)
X_train = X[:int(.9 * n_samples)]
y_train = y[:int(.9 * n_samples)]
X_test = X[int(.9 * n_samples):]
y_test = y[int(.9 * n_samples):]

# Define Lenet in Keras
keras_model = Sequential()
keras_model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(1,28,28), padding='same'))
keras_model.add(MaxPooling2D(pool_size=(2, 2)))
keras_model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
keras_model.add(MaxPooling2D(pool_size=(2, 2)))
keras_model.add(Flatten())
keras_model.add(Dense(512, activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(10, activation='softmax'))
keras_model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True))
keras_model.summary()

# Scale the input features
scale = 0.00390625
X_train = X_train*scale
X_test = X_test*scale



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 28, 28)        832       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 14, 14)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 14, 14)        51264     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 7, 7)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
__________

In [9]:
from systemml.mllearn import LogisticRegression

In [11]:
lr = LogisticRegression(spark)

In [12]:
lr.fit(X_train,y_train)


Welcome to Apache SystemML!
Version 1.3.0-SNAPSHOT
BEGIN MULTINOMIAL LOGISTIC REGRESSION SCRIPT
Reading X...
Reading Y...
-- Initially:  Objective = 10361.632918473206,  Gradient Norm = 4594.635433166934,  Trust Delta = 0.9415496143419578
-- Outer Iteration 1: Had 1 CG iterations, trust bound REACHED
   -- Obj.Reduction:  Actual = 3132.275489341813,  Predicted = 3177.5342460297334  (A/P: 0.9858),  Trust Delta = 1.7059852709010621
   -- New Objective = 7229.357429131393,  Beta Change Norm = 0.9415496143419577,  Gradient Norm = 4157.0562558858
 
-- Outer Iteration 2: Had 3 CG iterations, trust bound REACHED
   -- Obj.Reduction:  Actual = 2792.9810290719834,  Predicted = 3049.4211020466314  (A/P: 0.9159),  Trust Delta = 1.7059852709010621
   -- New Objective = 4436.376400059409,  Beta Change Norm = 1.7059852709010621,  Gradient Norm = 3622.7659629595482
 
-- Outer Iteration 3: Had 3 CG iterations, trust bound REACHED
   -- Obj.Reduction:  Actual = 1564.6880462436015,  Predicted = 1447.21

logReg

In [13]:
lr.score(X_test,y_test)

SystemML Statistics:
Total execution time:		0.006 sec.
Number of executed Spark inst:	0.




/home/spark/shared/user-libs/python3/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.906

In [17]:
prediction=lr.predict(X_test)

SystemML Statistics:
Total execution time:		0.005 sec.
Number of executed Spark inst:	0.




/home/spark/shared/user-libs/python3/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
ans = sum(prediction==y_test)

In [19]:
ans

453

In [21]:
len(y_test)

500

In [23]:
accuracy = (ans/len(y_test))*100

In [24]:
accuracy

90.60000000000001

In [25]:
from sklearn import datasets

In [26]:
data1 = datasets.load_iris()

In [27]:
data2 = datasets.load_diabetes()

In [28]:
data3 = datasets.load_breast_cancer()

In [29]:
X1 = data1.data
X2 = data2.data
X3 = data3.data

In [30]:
y1 = data1.target
y2 = data2.target
y3 = data3.target

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x_train1,x_test1,y_train1,y_test1 = train_test_split(X1,y1)
x_train2,x_test2,y_train2,y_test2 = train_test_split(X2,y2)
x_train3,x_test3,y_train3,y_test3 = train_test_split(X3,y3)

In [33]:
from systemml.mllearn import LogisticRegression

In [34]:
lg1 = LogisticRegression(spark)
lg2 = LogisticRegression(spark)
lg3 = LogisticRegression(spark)

In [35]:
lg1.fit(x_train1,y_train1)
lg2.fit(x_train2,y_train2)
lg3.fit(x_train3,y_train3)

BEGIN MULTINOMIAL LOGISTIC REGRESSION SCRIPT
Reading X...
Reading Y...
-- Initially:  Objective = 123.04457633082828,  Gradient Norm = 102.70108189411745,  Trust Delta = 0.1002167023689846
-- Outer Iteration 1: Had 1 CG iterations, trust bound REACHED
   -- Obj.Reduction:  Actual = 6.786774483145891,  Predicted = 6.477411741655935  (A/P: 1.0478),  Trust Delta = 0.14711745653882796
   -- New Objective = 116.25780184768239,  Beta Change Norm = 0.10021670236898458,  Gradient Norm = 86.30157193331232
 
-- Outer Iteration 2: Had 2 CG iterations, trust bound REACHED
   -- Obj.Reduction:  Actual = 7.6669055085015145,  Predicted = 7.556307437242339  (A/P: 1.0146),  Trust Delta = 0.18986958284696331
   -- New Objective = 108.59089633918087,  Beta Change Norm = 0.147117456538828,  Gradient Norm = 99.96760950841876
 
-- Outer Iteration 3: Had 2 CG iterations, trust bound REACHED
   -- Obj.Reduction:  Actual = 11.181578571060129,  Predicted = 11.24465342733129  (A/P: 0.9944),  Trust Delta = 0.3155

logReg

In [37]:
print(lg1.score(x_test1,y_test1))
print(lg2.score(x_test2,y_test2))
print(lg3.score(x_test3,y_test3))

SystemML Statistics:
Total execution time:		0.002 sec.
Number of executed Spark inst:	0.


0.9473684210526315


/home/spark/shared/user-libs/python3/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SystemML Statistics:
Total execution time:		0.006 sec.
Number of executed Spark inst:	0.


0.009009009009009009
SystemML Statistics:
Total execution time:		0.013 sec.
Number of executed Spark inst:	0.


0.9230769230769231


In [38]:
from systemml.mllearn import LinearRegression

In [39]:
lr = LinearRegression(spark)

In [40]:
lr.fit(x_train2,y_train2)

BEGIN LINEAR REGRESSION SCRIPT
Reading X and Y...
Running the CG algorithm...
||r|| initial value = 50348.06450391691,  target value = 0.050348064503916906
Iteration 1:  ||r|| / ||r init|| = 0.028676172087648947
Iteration 2:  ||r|| / ||r init|| = 0.008850106559744941
Iteration 3:  ||r|| / ||r init|| = 0.001989489952876567
Iteration 4:  ||r|| / ||r init|| = 6.114165574221006E-4
Iteration 5:  ||r|| / ||r init|| = 2.503233878293402E-4
Iteration 6:  ||r|| / ||r init|| = 6.786975406211641E-4
Iteration 7:  ||r|| / ||r init|| = 2.0259996770423416E-4
Iteration 8:  ||r|| / ||r init|| = 2.9813964155295545E-4
Iteration 9:  ||r|| / ||r init|| = 1.758927170108185E-4
Iteration 10:  ||r|| / ||r init|| = 2.9820012855746235E-4
Iteration 11:  ||r|| / ||r init|| = 1.433384562834574E-4
Iteration 12:  ||r|| / ||r init|| = 0.0011149405246487934
Iteration 13:  ||r|| / ||r init|| = 2.336296823795273E-9
The CG algorithm is done.
Computing the statistics...
AVG_TOT_Y,152.04833836858006
STDEV_TOT_Y,76.5899199512

lr

In [41]:
lr.score(x_test2,y_test2)

SystemML Statistics:
Total execution time:		0.002 sec.
Number of executed Spark inst:	1.




0.4822592593686483

In [42]:
lr.predict(x_test2)

SystemML Statistics:
Total execution time:		0.001 sec.
Number of executed Spark inst:	1.




array([[182.73192417],
       [115.94204766],
       [183.72421352],
       [107.48003533],
       [243.61546763],
       [119.36631011],
       [143.23358594],
       [150.20706972],
       [154.34240078],
       [255.9584285 ],
       [185.16659326],
       [ 62.05700685],
       [214.062156  ],
       [ 63.03470026],
       [123.41224294],
       [ 75.62859535],
       [233.70928108],
       [115.85962402],
       [118.12079067],
       [210.2954359 ],
       [244.03445906],
       [169.60826249],
       [240.55810476],
       [216.76112245],
       [ 81.39243794],
       [240.62070047],
       [144.44865442],
       [138.77209135],
       [ 70.64468403],
       [169.09278868],
       [181.56253836],
       [163.72070194],
       [ 58.34576658],
       [158.57310425],
       [ 92.46505329],
       [218.43486614],
       [105.34734135],
       [105.72227104],
       [101.53148952],
       [120.96660805],
       [171.96101047],
       [114.95142284],
       [102.28215303],
       [232

In [43]:
from systemml.mllearn import LogisticRegression

In [44]:
from mlxtend.data import mnist_data
import numpy as np
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Input, Dense
from keras import backend as K
from keras.models import Model
from keras.optimizers import SGD
# Set channel first layer
K.set_image_data_format('channels_first')

# Download the MNIST dataset
X, y = mnist_data()
X, y = shuffle(X, y)

# Split the data into training and test
n_samples = len(X)
X_train = X[:int(.9 * n_samples)]
y_train = y[:int(.9 * n_samples)]
X_test = X[int(.9 * n_samples):]
y_test = y[int(.9 * n_samples):]


In [45]:
lrg = LogisticRegression(spark)

In [46]:
lrg.fit(X_train,y_train)

BEGIN MULTINOMIAL LOGISTIC REGRESSION SCRIPT
Reading X...
Reading Y...
-- Initially:  Objective = 10361.632918473206,  Gradient Norm = 1180020.7227323812,  Trust Delta = 0.003686263414710899
-- Outer Iteration 1: Had 1 CG iterations, trust bound REACHED
   -- Obj.Reduction:  Actual = 3129.094244254142,  Predicted = 3188.4361073892605  (A/P: 0.9814),  Trust Delta = 0.0065674604212924695
   -- New Objective = 7232.538674219064,  Beta Change Norm = 0.003686263414710899,  Gradient Norm = 1084586.827637466
 
-- Outer Iteration 2: Had 3 CG iterations, trust bound REACHED
   -- Obj.Reduction:  Actual = 2754.310603623927,  Predicted = 3021.662667321488  (A/P: 0.9115),  Trust Delta = 0.0065674604212924695
   -- New Objective = 4478.228070595137,  Beta Change Norm = 0.0065674604212924695,  Gradient Norm = 939384.0898473396
 
-- Outer Iteration 3: Had 3 CG iterations, trust bound REACHED
   -- Obj.Reduction:  Actual = 1619.4077604714357,  Predicted = 1468.6050241949647  (A/P: 1.1027),  Trust Delt

logReg

In [49]:
prediction = lrg.predict(X_test)

SystemML Statistics:
Total execution time:		0.003 sec.
Number of executed Spark inst:	0.




/home/spark/shared/user-libs/python3/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
accuracy = (sum(prediction==y_test)/len(y_test))*100

In [51]:
accuracy

86.2

In [52]:
## Kaffe to dml work on the local env